## Rasta and OpenTripPlanner's REST API
Rasta has a module to postprocess data for navigation coming from the REST API of OpenTrip Planner (OTP). To use this, you should have basic understanding of OTP and also a running server will be needed to retrieve the data. I documented the server part and OTP in general [here](https://ikespand.github.io/posts/OpenTripPlanner/)

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import pkg_resources

Reconfirm that `keplergl_cli` you're using the keplergl_cli from a forked [GitHub](https://github.com/ikespand/keplergl_cli) instead of PyPi.

In [2]:
try:
    import keplergl_cli
    print("keplergl_cli is already installed... Checking version..")
    if not (pkg_resources.get_distribution('keplergl_cli').version == '0.3.3'):
        !pip install git+https://github.com/ikespand/keplergl_cli
    else:
        print("Good to go :)")
except ModuleNotFoundError:
    print("keplergl_cli is not available... Installing..")
    !pip install git+https://github.com/ikespand/keplergl_cli
import keplergl_cli
from rasta.navigate_with_otp import GetOtpRoute

keplergl_cli is already installed... Checking version..
Good to go :)


[Mapbox](https://account.mapbox.com/access-tokens/) is a company which provides the map tiles. To use it, you should have created the account there and receive an API key. Please remember, free service offers a generous amount of request which is more than enough for us.
Here, I saved my API key inside the .bashrc (.zshenv in zshell, therefore, importing it directly. Alternatively, you can copy and paste it here directly.

In [3]:
MAPBOX_API_KEY = os. environ['MAPBOX_API_KEY'] 
# MAPBOX_API_KEY = "MY_LONG_MAPBOX_API_KEY"

As mention, `rasta.navigate_with_otp` is the module which was developed to process the responce from the OpenTripPlanner. Remember, that your server is running for the OTP and you are querying in a region of your running server. 

In [23]:
my_otp_nav = GetOtpRoute(
    start_coord="48.81941, 9.27212",
    end_coord="48.78153, 9.20517",
    time="03:00:00", # Optinal, if not provided then it uses the current time
    date="2020/12/25", # Optinal, if not provided then it uses the current date
    MAPBOX_API_KEY=MAPBOX_API_KEY, # Necessary, only if you want to visualize in the map
    output_map_path="temporary_map", # To save the output kepler map
    mode="TRANSIT,WALK" # If not provided, then assuime the " Transit and walk"
)

In [24]:
gdf, html_path = my_otp_nav.extract_itinerary()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to temporary_map_vis.html!


/Users/ikespand/anaconda3/envs/rasta/lib/python3.7/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
/Users/ikespand/anaconda3/envs/rasta/lib/python3.7/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
/Users/ikespand/anaconda3/envs/rasta/lib/python3.7/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


Currently, it is hard-coded to have maximum 6 routes purposed by our module. It can be changed, if require.

In [25]:
print("Number of proposed routes by OTP:", len(gdf))
print("You can copy paste this query to browser to see the raw data:  ", my_otp_nav.address) 

Number of proposed routes by OTP: 3
You can copy paste this query to browser to see the raw data:   http://localhost:8080/otp/routers/default/plan?fromPlace=48.81941, 9.27212&toPlace=48.78153, 9.20517&time=03:00:00&date=2020/12/25&mode=TRANSIT,WALK


After having the output map, saved as HTML, we can also visualize directly inside the Jupyter. 

In [27]:
# Increase the map height from default 400 to 500
my_otp_nav.vis.map.height = 500 
my_otp_nav.vis.map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '3xb4acw', 'type': …

In [9]:
# Show one of the route in df
gdf[0].head()

,leg_mode,start_time,end_time,distance,duration,geometry
0,WALK,2020-10-25 13:00:00,2020-10-25 14:10:26,7897.694,4226.0,"LINESTRING (9.27208 48.81940, 9.27224 48.81900..."
